In [35]:
import websocket
import json
import pprint
import talib
import numpy as np
import config
import pandas as pd
from binance.client import Client
from binance.enums import *
import tkinter as tk
import asyncio

In [2]:
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@miniTicker"

In [3]:
# RSI_PERIOD = 14
# RSI_OVERBOUGHT = 70
# RSI_OVERSOLD = 30
# TRADE_SYMBOL = 'ETHUSD'
# TRADE_QUANTITY = 0.05

In [4]:
client = Client(config.API_KEY, config.API_SECRET, testnet = True)

In [5]:
# def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
#     try:
#         print("sending order")
#         order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
#         #print(order)
#     except Exception as e:
#         #print("an exception occurred - {}".format(e))
#         return False, e
#         #return "an exception occurred"

#     return True, order
#     #return order

In [6]:
# Putting in an order
#success, order_response = order('buy', '0.001', 'BTCUSDT')

In [7]:
# Printing order response
#pprint.pprint(order_response)

In [8]:
#res=client.get_exchange_info()


In [9]:
#print(client.response.headers)

In [10]:
#client.get_all_orders(symbol = 'BTCUSDT')

In [42]:
# Get 
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1627346625501,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.20640900', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '101.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2263.04887274', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [12]:
# balance_btc = client.get_asset_balance(asset='BTC')
# print(balance_btc)

In [13]:
def on_open(ws):
    print('opened connection')

In [14]:
def on_close(ws):
    print('closed connection')

In [15]:
# def on_message(ws, message):
#     print('received message')
#     json_message = json.loads(message)
#     pprint.pprint(json_message)
    
#     candle = json_message['k']
    
#     is_candle_closed = candle['x']
#     close = candle['c']
    
#     if is_candle_closed:
#         print("candle closed at {}".format(close))
#         closes.append(float(close))
#         print("closes")
#         print(closes)
        
#         if len(closes) > RSI_PERIOD

In [16]:
financials_df = pd.DataFrame(columns = ['close', 'bollinger_signal'])
financials_df.head()

,close,bollinger_signal


In [17]:
BOLLINGER_WINDOW = 20
def get_bollinger_signal(close):
    #print('the close price is ')
    #print(close)
    global financials_df
    #print(financials_df)
    #print(financials_df['close'])
    if len(financials_df) < BOLLINGER_WINDOW:
        financials_df = financials_df.append({'close' : float(close), 'bollinger_signal' : 0}, ignore_index = True)
        return
    bollinger_mean = financials_df['close'].rolling(window=BOLLINGER_WINDOW).mean()
    #print('bollinger mean is ')
    #print(bollinger_mean)
    bollinger_std = financials_df['close'].rolling(window=BOLLINGER_WINDOW).std()
    #print('bollinger std is ')
    #print(bollinger_std)
    bollinger_upper_band = bollinger_mean + bollinger_std
    #print('bollinger upper band is ')
    #print(bollinger_upper_band)
    bollinger_lower_band = bollinger_mean - bollinger_std
    #print('bollinger lower band is ')
    #print(bollinger_lower_band)
    bollinger_long = np.where(close < bollinger_lower_band, 1.0, 0.0)
    #print('bollinger long is ')
    #print(bollinger_long)
    bollinger_short = np.where(close > bollinger_upper_band, -1.0, 0.0)
    #print('bollinger short is ')
    #print(bollinger_short)
    bollinger_signal = bollinger_long + bollinger_short
    #print('bollinger signal is ')
    #print(bollinger_signal)
    financials_df = financials_df.append({'close' : float(close), 'bollinger_signal' : bollinger_signal[-1]}, ignore_index = True)


    return

In [18]:
#get_bollinger_signal(42000)

In [19]:
#bollinger_mean = financials_df['close'].rolling(window=BOLLINGER_WINDOW).mean()
#print(bollinger_mean)

In [20]:
#financials_df

In [21]:
def on_message(ws, message):
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)
    close = json_message['c']
    #print('close price ' + close)
    # get Bollinger values
    #print('financials df length ' + len(financials_df))
    get_bollinger_signal(float(close))
    #print('bollinger signal ' + bollinger_signal)
    #financials_df = financials_df.append({'close' : float(close), 'bollinger_signal' : bollinger_signal})
    #print("close")
    #print('got bollinger signal')

In [22]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)

In [23]:
ws.run_forever()

opened connection


False

In [24]:
financials_df

,close,bollinger_signal
0,37666.35,0.0
1,37669.53,0.0
2,37669.33,0.0
3,37686.75,0.0
4,37690.71,0.0
5,37699.00,0.0
6,37694.65,0.0
7,37694.66,0.0
8,37694.66,0.0
9,37694.66,0.0


In [32]:
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        #print(order)
    except Exception as e:
        #print("an exception occurred - {}".format(e))
        return False, e
        #return "an exception occurred"

    return True, order
    #return order

In [39]:
if financials_df['bollinger_signal'].iloc[-1] > 0:
    order('buy', '0.001', 'BTCUSDT')
elif financials_df['bollinger_signal'].iloc[-1] < 0:
    order('sell', '0.001', 'BTCUSDT')
else:
    print('hold')

sending order


In [43]:
# current BTC balance = 1.20840900
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1627346625501,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.20640900', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '101.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2263.04887274', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}